In [1]:
import os
import numpy as np
import pydicom
import csv
from skimage.draw import polygon
from scipy.ndimage import binary_fill_holes

# Function to create a binary mask from an RT structure set
def create_binary_mask(rtstruct, roi_number, reference_image_shape, spacing, origin):
    """
    Create a binary mask for a specific ROI in the RT structure set.
    """
    contours = []
    for roi_contour in rtstruct.ROIContourSequence:
        if roi_contour.ReferencedROINumber == str(roi_number):
            if hasattr(roi_contour, "ContourSequence") and roi_contour.ContourSequence:
                for contour_seq in roi_contour.ContourSequence:
                    contour_data = contour_seq.ContourData
                    contour = np.array(contour_data).reshape(-1, 3)  # Convert to (x, y, z)
                    contours.append(contour)

    # Create an empty binary mask
    mask = np.zeros(reference_image_shape, dtype=np.uint8)

    for contour in contours:
        # Map the contour to the voxel grid
        z = int(np.round((contour[0, 2] - origin[2]) / spacing[2]))
        x = np.round((contour[:, 0] - origin[0]) / spacing[0]).astype(int)
        y = np.round((contour[:, 1] - origin[1]) / spacing[1]).astype(int)
        rr, cc = polygon(y, x, mask[z].shape)
        mask[z, rr, cc] = 1

    # Fill holes in the mask
    for z in range(mask.shape[0]):
        mask[z] = binary_fill_holes(mask[z])

    return mask

# Function to calculate Dice coefficient
def calculate_dice(mask1, mask2):
    intersection = np.sum(np.logical_and(mask1, mask2))
    union = np.sum(mask1) + np.sum(mask2)
    return 2 * intersection / union if union != 0 else 1.0

# Function to process and compare two RTSS
def compare_rt_dicom(rt_file1, rt_file2, ref_image_shape, spacing, origin, output_csv_path):
    rtstruct1 = pydicom.dcmread(rt_file1)
    rtstruct2 = pydicom.dcmread(rt_file2)

    # Map ROI names to numbers
    roi_map1 = {roi.ROIName: roi.ROINumber for roi in rtstruct1.StructureSetROISequence}
    roi_map2 = {roi.ROIName: roi.ROINumber for roi in rtstruct2.StructureSetROISequence}

    # Find matching ROI names
    matching_rois = set(roi_map1.keys()).intersection(set(roi_map2.keys()))

    results = []

    for roi_name in matching_rois:
        roi_number1 = roi_map1[roi_name]
        roi_number2 = roi_map2[roi_name]

        # Generate binary masks for the matching ROIs
        mask1 = create_binary_mask(rtstruct1, roi_number1, ref_image_shape, spacing, origin)
        mask2 = create_binary_mask(rtstruct2, roi_number2, ref_image_shape, spacing, origin)

        # Calculate Dice coefficient
        dice = calculate_dice(mask1, mask2)
        results.append([roi_name, dice])

    # Save results to CSV
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["ROI", "DiceCoefficient"])
        writer.writerows(results)

    print(f"Results saved to {output_csv_path}")

In [3]:
def get_reference_image_shape(dicom_folder):
    """
    Determine the shape of the reference image from a folder of DICOM files.
    
    Parameters:
        dicom_folder (str): Path to the folder containing DICOM image slices.
    
    Returns:
        tuple: (Rows, Columns, Number of Slices)
    """
    # List all DICOM files in the folder
    dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith('.dcm')]

    if not dicom_files:
        raise ValueError("No DICOM files found in the specified folder.")

    # Read the first DICOM file to get Rows and Columns
    first_dicom = pydicom.dcmread(dicom_files[0])
    rows = first_dicom.Rows
    columns = first_dicom.Columns

    # Count the number of slices (unique InstanceNumber or ImagePositionPatient[2])
    slice_positions = []
    for dicom_file in dicom_files:
        ds = pydicom.dcmread(dicom_file)
        if hasattr(ds, "ImagePositionPatient"):
            slice_positions.append(ds.ImagePositionPatient[2])
        elif hasattr(ds, "InstanceNumber"):
            slice_positions.append(ds.InstanceNumber)

    num_slices = len(set(slice_positions))

    return (rows, columns, num_slices)

In [4]:
def get_spacing_and_origin(dicom_folder):
    """
    Determine the spacing and origin of the reference image from a folder of DICOM files.
    
    Parameters:
        dicom_folder (str): Path to the folder containing DICOM image slices.
    
    Returns:
        tuple: (Spacing (row, column, slice), Origin (x, y, z))
    """
    # List all DICOM files in the folder
    dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith('.dcm')]

    if not dicom_files:
        raise ValueError("No DICOM files found in the specified folder.")

    # Read the first DICOM file
    first_dicom = pydicom.dcmread(dicom_files[0])

    # Extract pixel spacing (row, column)
    pixel_spacing = first_dicom.PixelSpacing  # [row spacing, column spacing]

    # Extract slice thickness or spacing between slices
    slice_thickness = getattr(first_dicom, "SpacingBetweenSlices", None) or getattr(first_dicom, "SliceThickness", 1.0)

    # Extract the origin (ImagePositionPatient)
    origin = first_dicom.ImagePositionPatient  # [x, y, z]

    # Combine to get spacing (row, column, slice)
    spacing = (pixel_spacing[0], pixel_spacing[1], slice_thickness)

    return spacing, origin

# Specify the folder containing DICOM image slices
root = "D:/Datasets/CT_Recon/Breast/"
IDs = os.listdir(root)
idx = 0
AI_manual_folder = f"{root}{IDs[idx]}/Aice/Onco_Aice/"
IR_manual_folder = f"{root}{IDs[idx]}/AIDR/Onco_AIDR/"

# Get the reference image shape
image_shape = get_reference_image_shape(AI_manual_folder)
print(f"Reference Image Shape: {image_shape}")

# Get the spacing and origin
spacing, origin = get_spacing_and_origin(IR_manual_folder)
print(f"Spacing: {spacing}")
print(f"Origin: {origin}")

Reference Image Shape: (512, 512, 94)
Spacing: ('1.212', '1.212', '5')
Origin: [-309.745, -309.7454, -1236.5]


In [5]:
AI_manual = f"{AI_manual_folder}{os.listdir(AI_manual_folder)[-1]}"
IR_manual = f"{IR_manual_folder}{os.listdir(IR_manual_folder)[-1]}"
# print(AI_manual)
# print(IR_manual)
output_csv = "./dice_coefficients.csv"

Dice for Liver: 1.0
